# 🔍 QualiVault: Validate Transcripts
**Goal:** Use Ollama (local LLM) to detect transcription errors and hallucinations.

1. Loads transcripts from CSV files.
2. Samples segments and sends them to Ollama for validation.
3. Flags potential errors: hallucinations, misheard words, artifacts.
4. Generates validation report with suggestions.

**Prerequisites:**
- Ollama must be installed and running (`ollama serve`)
- Install a model: `ollama pull llama3.1` or `ollama pull jobautomation/OpenEuroLLM-Danish:latest`


In [ ]:
%load_ext autoreload
%autoreload 2
import yaml
from pathlib import Path
from qualivault.validation import OllamaValidator, validate_recipe_transcripts
from qualivault import find_workspace_root

# ============================================
# PROJECT CONFIGURATION
# ============================================
# Specify your project folder name here:
PROJECT_NAME = 'YOUR_PROJECT_NAME'  # <-- Change this to your project folder name

# Auto-detect workspace root and project path
workspace_root = find_workspace_root(Path.cwd())
project_root = workspace_root / 'projects' / PROJECT_NAME

# Verify project exists
if not project_root.exists():
    raise FileNotFoundError(
        f"❌ Project not found: {project_root}\n   Available projects in {workspace_root / 'projects'}:"
    )

config_path = project_root / 'config.yml'
if not config_path.exists():
    raise FileNotFoundError(f"❌ Config not found: {config_path}")

print(f"🎯 Working on project: {PROJECT_NAME}")
print(f"📁 Project root:       {project_root}")
print(f"⚙️  Config file:        {config_path}")
print()

# 1. Load Configuration
with open(config_path, encoding='utf-8') as f:
    config = yaml.safe_load(f)

recipe_path = project_root / "processing_recipe.yaml"
transcripts_dir = (project_root / config['paths']['output_base_folder']).resolve()

print(f"📂 Transcripts: {transcripts_dir}")
print(f"📋 Recipe: {recipe_path}")


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


FileNotFoundError: ❌ Project not found: c:\dev\qualvalt\projects\YOUR_PROJECT_NAME
   Available projects in c:\dev\qualvalt\projects:

In [ ]:
# 2. Check Ollama Installation and Available Models
import requests
import json

OLLAMA_URL = "http://localhost:11434"

def check_ollama():
    """Test if Ollama is running and list available models."""
    print("🔍 Checking Ollama installation...\n")
    
    # Test connection
    try:
        response = requests.get(f"{OLLAMA_URL}/api/tags", timeout=5)
        
        if response.status_code == 200:
            print("✅ Ollama is running!")
            
            # List available models
            data = response.json()
            models = data.get('models', [])
            
            if models:
                print(f"\n📦 Available models ({len(models)}):\n")
                
                for model in models:
                    name = model.get('name', 'Unknown')
                    size_gb = model.get('size', 0) / (1024**3)
                    
                    # Highlight Danish model
                    if 'danish' in name.lower() or 'openeuro' in name.lower():
                        print(f"   🇩🇰 {name} ({size_gb:.1f} GB) ← Recommended for Danish")
                    elif 'llama3' in name.lower():
                        print(f"   🦙 {name} ({size_gb:.1f} GB) ← Good general model")
                    else:
                        print(f"   • {name} ({size_gb:.1f} GB)")
                
                print("\n💡 Recommendation:")
                danish_models = [m for m in models if 'danish' in m.get('name', '').lower() or 'openeuro' in m.get('name', '').lower()]
                
                if danish_models:
                    print(f"   Use: '{danish_models[0]['name']}' (Danish-optimized)")
                elif any('llama3' in m.get('name', '').lower() for m in models):
                    llama3 = [m for m in models if 'llama3' in m.get('name', '').lower()][0]
                    print(f"   Use: '{llama3['name']}' (general purpose)")
                else:
                    print(f"   Use: '{models[0]['name']}'")
                
                return True, models
            else:
                print("⚠️  Ollama is running but no models are installed!")
                print("\n📥 Install a model:")
                print("   For Danish: ollama pull jobautomation/OpenEuroLLM-Danish:latest")
                print("   General:    ollama pull llama3.1")
                return False, []
        else:
            print(f"❌ Ollama responded with error: {response.status_code}")
            return False, []
            
    except requests.exceptions.ConnectionError:
        print("❌ Cannot connect to Ollama!")
        print("\n🔧 To fix:")
        print("   1. Install Ollama: https://ollama.ai")
        print("   2. Start Ollama: ollama serve")
        print("   3. Pull a model: ollama pull llama3.1")
        return False, []
    except Exception as e:
        print(f"❌ Error checking Ollama: {e}")
        return False, []

# Run the check
ollama_ok, available_models = check_ollama()


## Configuration

Adjust these settings to control validation:

- **model**: Ollama model to use (`llama2`, `mistral`, `llama3`, etc.)
- **sample_rate**: Fraction of segments to check (0.1 = 10%, 1.0 = 100%)
- **language**: Expected language of transcripts
- **ollama_url**: URL where Ollama is running (default: localhost)

In [ ]:
# Validation Settings
# Auto-select the best available model from the list above
if ollama_ok and available_models:
    # Prefer Danish models, then llama3, then first available
    danish_models = [m for m in available_models if 'danish' in m.get('name', '').lower() or 'openeuro' in m.get('name', '').lower()]
    llama3_models = [m for m in available_models if 'llama3' in m.get('name', '').lower()]
    
    if danish_models:
        MODEL = danish_models[0]['name']
    elif llama3_models:
        MODEL = llama3_models[0]['name']
    else:
        MODEL = available_models[0]['name']
else:
    MODEL = "llama3.1"  # Fallback if check didn't run

SAMPLE_RATE = 0.1           # Check 10% of segments (faster, set to 1.0 for 100%)
LANGUAGE = "Danish"         # Expected language
OLLAMA_URL = "http://localhost:11434"
TIMEOUT = 120               # Timeout in seconds (increase for large models)

VALIDATION_PARAMS = {
    "sample_rate": SAMPLE_RATE,
    "min_text_length": 8,
    "max_segments": None,
    "timeout": TIMEOUT,
    "ollama_options": {"temperature": 0.3, "top_p": 0.9}
}

print(f"🤖 Model: {MODEL}")
print(f"📊 Sample Rate: {SAMPLE_RATE * 100}%")
print(f"🌍 Language: {LANGUAGE}")
print(f"⏱️  Timeout: {TIMEOUT}s")


## Test Ollama Connection

Make sure Ollama is running before proceeding.

In [ ]:
validator = OllamaValidator(model=MODEL, ollama_url=OLLAMA_URL)

# Quick test
test_response = validator._query_ollama("Say 'Hello' in one word.")
if test_response:
    print(f"✅ Ollama is responding: '{test_response.strip()[:50]}'")
else:
    print("❌ Ollama is not responding. Make sure it's running: `ollama serve`")

## Validate All Transcripts

This will:
1. Load all transcribed interviews from the recipe
2. Sample segments from each CSV
3. Check each segment with Ollama for errors
4. Save individual validation report for each interview
5. Create summary statistics


In [ ]:
# Run validation on all transcripts - Save individual report for each interview
from datetime import datetime

from qualivault import validate_transcripts_to_individual_reports

# RESUME OPTION: Set to True to re-validate all files (overwrites existing reports)
FORCE_REVALIDATE = False  # Set to True to start fresh, False to resume from last position

print(f"🔍 Starting validation at {datetime.now().strftime('%H:%M:%S')}")
print(f"   Model: {MODEL}")
print(f"   Sample Rate: {VALIDATION_PARAMS['sample_rate'] * 100}%")
print(f"   Max Interviews: {VALIDATION_PARAMS.get('max_interviews') or 'All'}")
print(f"   Resume Mode: {'OFF (re-validate all)' if FORCE_REVALIDATE else 'ON (skip existing)'}")
print(f"   Scanning: {transcripts_dir}")
print("=" * 70)

reports_dir = project_root / "validation_reports"

all_results = validate_transcripts_to_individual_reports(
    transcripts_dir=transcripts_dir,
    reports_dir=reports_dir,
    model=MODEL,
    ollama_url=OLLAMA_URL,
    language=LANGUAGE,
    validation_params=VALIDATION_PARAMS,
    resume=(not FORCE_REVALIDATE),
    max_files=VALIDATION_PARAMS.get("max_interviews"),
)

# Backwards-compat variable name for older cells
results = all_results

print("\n" + "=" * 70)
print(f"✅ Validation complete at {datetime.now().strftime('%H:%M:%S')}")
print(f"   Total with reports: {len(all_results)}")
print(f"   Reports saved to: {reports_dir}")


## Review Validation Reports

Inspect flagged segments and issues across all transcripts.


In [ ]:
# Summary Statistics - Load all individual reports and aggregate
from qualivault import (
    aggregate_validation_reports,
    load_validation_reports,
)

reports_dir = project_root / "validation_reports"
all_results = load_validation_reports(reports_dir)

print(f"📂 Loading {len(all_results)} validation reports\n")

df, totals = aggregate_validation_reports(reports_dir=reports_dir, all_results=all_results)

total_checked = totals["total_segments_checked"]
total_flagged = totals["total_segments_flagged"]

print(f"📊 VALIDATION SUMMARY")
print(f"=" * 70)
print(f"Files validated: {totals['files_validated']}")
print(f"Total segments checked: {total_checked}")
print(f"Total segments flagged: {total_flagged}")
if total_checked > 0:
    print(f"Flag rate: {(total_flagged / total_checked * 100):.1f}%")
print(f"=" * 70)

print("\n📁 Per-File Results:")
print(df.to_string(index=False))

print(f"\n💾 Summary saved to: {reports_dir / 'SUMMARY.csv'}")


## Detailed Issue Review

Examine specific flagged segments.

In [ ]:
# Show detailed issues for first transcript
from qualivault import load_validation_reports

try:
    all_results
except NameError:
    all_results = load_validation_reports(project_root / "validation_reports")

if all_results and all_results[0].get("flagged_segments"):
    report = all_results[0]
    print(f"\n🔍 Detailed issues for: {report.get('csv_file', 'unknown')}\n")

    for seg in report["flagged_segments"][:10]:  # Show first 10
        print(f"Segment {seg['segment_index']} ({seg['start']:.1f}s - {seg['end']:.1f}s)")
        print(f"Speaker: {seg['speaker']}")
        print(f"Text: {seg['text']}")
        print(f"Issues: {', '.join(seg['issues'])}")
        print(f"Confidence: {seg['confidence']:.2f}")
        if seg.get("suggestions"):
            print(f"Suggestions: {seg['suggestions']}")
        print()
else:
    print("No issues found in first transcript.")


## Export Validation Report

Create master summary file with links to individual validation reports.


In [ ]:
# Create Master Summary Report
from qualivault import (
    load_validation_reports,
    write_master_validation_summary,
)

reports_dir = project_root / "validation_reports"

try:
    all_results
except NameError:
    all_results = load_validation_reports(reports_dir)

summary_file = write_master_validation_summary(
    reports_dir=reports_dir,
    all_results=all_results,
    model=MODEL,
    sample_rate=SAMPLE_RATE,
    language=LANGUAGE,
)

print(f"📋 Master Summary Report")
print(f"=" * 70)
print(f"📁 Location: {summary_file}")
print(f"   • Master summary: {summary_file.name}")
print(f"   • Individual reports: {len(list(reports_dir.glob('*_validation.json')))} files")
print(f"   • Summary CSV: SUMMARY.csv")
